In [ ]:
# 단어 자동 완성 만들기
# RNN 모델을 이용하여 단어를 자동 완성하는 프로그램을 만들어 보자.
# 영문자 4개로 구성된 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를
# 완성하는 프로그램이다.
# 학습시킬 데이터는 영문자로 구성된 임의의 단어를 사용한 것이고, 한 글자 한 글자를 하나의
# 단계로 볼 것이다. 그러면 한 글자가 한 단계의 입력값이 되고, 총 글자 수가 전체 단계가
# 된다.

import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# 알파벳 글자 순서에 해당하는 인덱스 번호와 글자를 딕셔너리에 저장
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n:i for i, n in enumerate(char_arr)}
print(num_dic)
dic_len = len(num_dic)    
print(dic_len)            # 26

# 학습에 사용할 단어를 배열로 저장
# 다음 배열은 입력갑과 출력값으로 다음처럼 사용할 것이다.
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word','wodd','deep','dive','cold','load','love','kiss','kind']

# 학습에 사용할 단어들을 one-hot-encoding으로 변환하는 함수
def make_batch(seq_date):
  input_batch = []  # 학습에 사용할 단어의 처음 세글자의 알파벳 인덱스 번호를 저장할 배열
  target_batch = [] # 학습에 사용할 단어의 마지막 글자의 알파벳 인덱스 번호를 저장할 배열

  for seq in seq_data:
    # 1. 입력값용으로, 단어의 처름 세 글자의 알파벳 인덱스를 구한 배열을 만든다.
    # [22,14,17][22,14,14][3,4,4][3,8,21]...
    input = [num_dic[n] for n in seq[:-1]]

    # 2. 출력값용으로, 마지막 글자의 알파벳 인덱스를 구한다.
    # 3,3,15,4,3,...
    target = num_dic[seq[-1]]

    # 3. 입력된값을 원-핫 인코딩으로 변환한다.
    # [[ 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    # [ 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
    # [ 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
    input_batch.append(np.eye(dic_len)[input])

    target_batch.append(target)

  return input_batch, target_batch

#############
# 옵션 설정
#############
learning_rate = 0.01      # 학습률
n_hidden =128             # 은닉층의 노드 갯수
total_epoch = 30          # 학습 횟수

# 단어의 전체 중 처음 3글자를 단계적으로 학습할 것이므로, n_step은 3이 된다.
# 타입 스텝: [1,2,3] => 3
# RNN을 구성하는 시퀀스의 갯수
n_step = 3

# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개 가 된다.
# c => [0 0 1 0 0 0 0 0 ...0]
# 출력값도 입력값과 마차나지로 26개의 알파벳으로 분류한다.
n_input = n_class = dic_len

###################
# 신경망 모델 구성
###################
X = tf.placeholder(tf.float32, [None, n_step, n_input])

# 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용하므로
# 출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용하기 때문에
# 다음처럼 하나의 값만 있는 1차원 배열을 입력값으로 받는다.
# [3] [3] [15] [4] ...
# 기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 여야한다.
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))


# RNN셀을 생성
# 여러 셀을 조합해 심층 신경망을 만들기 위해서 2개의 RNN 셀을 생성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 과적합 방지를 위한 Dropout 기법을 사용
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)

# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성한다.
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 여러개의 셀을 조합한 RNN셀을 생성
# 위에서 만든 2개의 RNN셀을 MultiRNNCell() 함수를 사용하여 조합한다.
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell1])

# tf.nn.dynamic_rnn 함수를 이용해 심층 순환 신경망, 즉 Deep RNN을 만든다.
# time_major = True
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 출력층
# 최종 결과는 one-hot 인코딩 형식으로 만듭니다.
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# 손실 함수로는 sparse_sofrmax_cross_entropy_with_logits 를,
# 최적화 함수로는 AdamOptimizer를 설멍하여 신경망 모델 구성을 마무리 한다.
cost = tf.reduce_mean(
          tf.nn.sparse_sofrmax_cross_entropy_with_logits(
              logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

####################
# 신경망 모델 학습
####################
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# make_batch 함수를 이용하여 seq_data에 저장한 단어들을 입력값(처음 세 글자)과 실측값(마지막 한 글자)으로
# 분리하고, 이 값들을 최적화 함수를 실행하는 코드에 넣어 신경망를 학습시킨다. 
input_batch, target_batch = make_batch(seq_data)

# 30회 학습
for epoch in range(total_epoch):
  _, loss = sess.run([optimizer, cost],
                  feed_dict = {X: input_batch, Y:target_batch})
  print('Epoch:', '%04d' %(epoch + 1), 'cost= ', '{:.6f}'.format(loss))

print('최적화 완료')

##############
# 결과 확인
##############
# 레이블 값이 정수이므로 예측값도 정수로 변경해준다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)

# one-hot 인코딩이 아니므로 입력값을 그대로 비교한다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

# 학습에 사용한 단어들을 넣고 예측 모델을 돌린다.
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dick={X: input_batch, Y: target_batch})

# 모델이 예측한 값들을 가지고, 각각의 값에 해당하는 인덱스의 알파벳을 가져와서 예측한 단어를 출력
predict_words = []
for idx, val in enumerate(seq_data):
  last_char = char_arr[predict[idx]]
  predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

# replaceholder 부분 수정 필요 (74, 81)